In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog, messagebox, simpledialog
import pandas as pd
from webdav4.client import Client
import os
from getmac import get_mac_address
import pandas as pd
from datetime import datetime
import openpyxly


In [6]:
try:
    df = pd.read_excel('glossary.xlsx', engine='openpyxl')
    glossary = {}
    for index, row in df.iterrows():
        term = str(row['terms']).upper()
        full = str(row['full_form'])
        expl = str(row['explanation'])
        if term:
            glossary[term] = {"full_form": full, "explanation": expl}
except FileNotFoundError:
    # 创建一个空的DataFrame
    df = pd.DataFrame(columns=['terms', 'full_form', 'explanation'])
    # 将DataFrame保存为Excel文件
    df.to_excel('glossary.xlsx', index=False)
    # 创建一个空的词典
    glossary = {}

print (glossary)

{'QC': {'full_form': 'Quantum Computer', 'explanation': '量子计算机'}}


In [ ]:
def search_term(event=None):
    search_query = entry_search.get().lower()
    matching_terms = []

    # 清空文本框
    text_full_form.delete(1.0, tk.END)
    text_explanation.delete(1.0, tk.END)

    # 在术语表中进行模糊搜索
    for term, data in glossary.items():
        full_form_term = data["full_form"].lower()
        explanation_term = data["explanation"].lower()
        
        if search_query in term.lower() or search_query in full_form_term or search_query in explanation_term:
            matching_terms.append(term)
    
    if matching_terms:
        result = glossary[matching_terms[0]]
        # 插入新内容
        text_full_form.insert(tk.END, result["full_form"])
        text_explanation.insert(tk.END, result["explanation"])

        # 更新左侧列表的选中项，并确保它在可视范围内
        index = glossary_list.get(0, tk.END).index(matching_terms[0])  #获取匹配术语在术语列表中的索引
        glossary_list.selection_clear(0, tk.END)
        glossary_list.selection_set(index)
        glossary_list.see(index)
    else:
        messagebox.showerror("Error", "Term not found")

In [4]:
 # 更新标题栏中的术语数目
def update_title():
    root.title(f"Glossary ({current_version}) - {len(glossary)} terms")


TclError: no display name and no $DISPLAY environment variable

In [ ]:
# 更新术语列表
def update_glossary_list():
    glossary_list.delete(0, tk.END)  # 清空术语列表
    
    for term in sorted(glossary.keys()):  # 按字母顺序排序
        glossary_list.insert(tk.END, term)

    glossary_count.set(len(glossary))

In [39]:
# 添加术语
def add_term(event=None):  # 修改函数签名以接受事件参数，参数设置为None以确保不会影响直接调用
    term = "t"
    full = "test"
    expl = "tt"
    if term:
        glossary[term] = {"full_form": full, "explanation": expl}
        # 更新Excel文件
        # df = pd.DataFrame(list(glossary.items()), columns=['full_form', 'explanation'])################## 1st error

        # 将字典转换为 DataFrame，并设置合适的列名
        df = pd.DataFrame.from_dict(glossary, orient='index')
        df.index.name = 'terms'
        df.reset_index(inplace=True)
        # 将 DataFrame 写入 Excel 文件
        df.to_excel('glossary.xlsx', index=False)
        print ("Success", "Term added successfully")
        # update_glossary_list()  # 更新术语列表
        # update_title()  # 更新标题栏中的术语数目
    else:
        print ("Error", "Please fill in the term field")

add_term()
print (glossary)

Success Term added successfully
{'QC': {'full_form': 'Quantum Computer', 'explanation': '量子计算机'}, 't': {'full_form': 'test', 'explanation': 'tt'}}


In [28]:
# 删除术语
def delete_term():
    # selection = glossary_list.curselection()  # 获取当前选中的索引
    # if selection:
        # term = glossary_list.get(selection[0])  # 获取当前选中的术语
        term = "t"
        if term in glossary:
            del glossary[term]
            # 更新Excel文件
            df = pd.DataFrame(list(glossary.items()), columns=['full_form', 'explanation'])
            df.to_excel('glossary.xlsx', index=False)
            print ("Success", "Term deleted successfully")
            # update_glossary_list()  # 更新术语列表
            # update_title()  # 更新标题栏中的术语数目
        else:
            print ("Error", "Term not found")

delete_term()
print (glossary)

Error Term not found
{'QC': {'full_form': 'Quantum Computer', 'explanation': '量子计算机'}}


In [38]:
# 更新术语
def update_term():
    # selection = glossary_list.curselection()  # 获取当前选中的索引
    # if selection:
    #     term = glossary_list.get(selection[0])  # 获取当前选中的术语
        term = "t"
        if term in glossary:
            # full = simpledialog.askstring("Update Term", "Enter the full form:", initialvalue=glossary[term]['full_form'])
            full = "test"
            expl = "tt"
            # Check if user pressed Cancel
            if full is None:
                return

            # expl = simpledialog.askstring("Update Term", "Enter the explanation:", initialvalue=glossary[term]['explanation'])
            
            # Check if user pressed Cancel
            if expl is None:
                return

            if full and expl:
                glossary[term] = {"full_form": full, "explanation": expl}
                # 更新Excel文件
                df = pd.DataFrame(list(glossary.items()), columns=['full_form', 'explanation'])
                df.to_excel('glossary.xlsx', index=False)
                print ("Success", "Term updated successfully")
                # update_glossary_list()  # 更新术语列表
            else:
                print ("Error", "Please fill in all fields")
        else:
            print ("Error", "Term not found")

update_term()
print (glossary)

Success Term updated successfully
{'QC': {'full_form': 'Quantum Computer', 'explanation': '量子计算机'}, 't': {'full_form': 'test', 'explanation': 'tt'}}


In [ ]:
# 显示术语
def show_term(event):
    selection = glossary_list.curselection()
    if selection:
        term = glossary_list.get(selection[0])
        result = glossary[term]
        
        # 清空文本框
        text_full_form.delete(1.0, tk.END)
        text_explanation.delete(1.0, tk.END)

        # 插入新内容
        text_full_form.insert(tk.END, result["full_form"])
        text_explanation.insert(tk.END, result["explanation"])

In [ ]:
# 显示右键菜单        
def show_context_menu(event):
    global context_menu  # 将context_menu声明为全局变量
    selection = glossary_list.curselection()
    if selection:
        context_menu = tk.Menu(glossary_list, tearoff=0)
        context_menu.add_command(label="Update", command=update_term)
        context_menu.add_command(label="Delete", command=delete_term)
        context_menu.post(event.x_root, event.y_root)

In [ ]:
# 上传文件到坚果云
def upload_to_jianguoyun():
    # 检查本地是否存在glossary文件
    if not os.path.exists('glossary.xlsx'):
        print("Error: Glossary file does not exist")
        return

    # 如果存在，继续执行上传操作
    mac_address = get_mac_address()  # 获取设备的 MAC 地址
    cloud_filename = f'{mac_address.replace(":", "")}.xlsx'

    # 坚果云账号和密码
    username = "imybfu@outlook.com"
    password = "axzyqd3a6brx99ei"

    client = Client(base_url='https://dav.jianguoyun.com/dav/', auth=(username, password))

    # 上传文件到指定路径
    cloud_path = f'/nutshare/glossary/{cloud_filename}'  # 替换为您的实际目标路径
    try:
        client.upload_file(from_path='glossary.xlsx', to_path=cloud_path, overwrite=True)
        print("Success: File uploaded successfully")
    except Exception as e:
        print(f"Error: Failed to upload file: {str(e)}")

In [ ]:
# 从坚果云下载文件
def download_from_jianguoyun():
    # 生成当前时间的标记，用于重命名本地文件
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    
    # 备份本地文件
    backup_filename = f"glossary_{current_time}.xlsx"
    try:
        # 如果本地有glossary文件，将之改名为备份文件
        if os.path.exists("glossary.xlsx"):
            os.rename("glossary.xlsx", backup_filename)
    except Exception as e:
        print(f"Error: Failed to backup local file: {str(e)}")
        return

    mac_address = get_mac_address()  # 获取设备的 MAC 地址
    cloud_filename = f'{mac_address.replace(":", "")}.xlsx'

    # 坚果云账号和密码
    username = "imybfu@outlook.com"
    password = "axzyqd3a6brx99ei"

    client = Client(base_url='https://dav.jianguoyun.com/dav/', auth=(username, password))

    # 下载远程文件到本地
    cloud_path = f'/nutshare/glossary/{cloud_filename}' # 替换为您的实际文件路径
    try:
        client.download_file(from_path=cloud_path, to_path="glossary.xlsx")
        # 如果下载成功，则更新术语列表和标题栏，并显示成功消息
        try:
            df = pd.read_excel('glossary.xlsx', engine='openpyxl')  # 重新读取下载后的 glossary.xlsx 文件
            glossary.clear()  # 清空当前的 glossary 字典
            for index, row in df.iterrows():
                term = str(row['terms']).upper()
                full = str(row['full_form'])
                expl = str(row['explanation'])
                if term:
                    glossary[term] = {"full_form": full, "explanation": expl}  # 更新 glossary 字典
        except FileNotFoundError:
            print("Error: Failed to load downloaded glossary file")
            return

        update_glossary_list()
        update_title()
        print("Success: File downloaded successfully")

    except Exception as e:
        print(f"Error: Failed to download file: {str(e)}")
        # 下载失败，将备份文件名改回glossary
        if os.path.exists(backup_filename):
            os.rename(backup_filename, "glossary.xlsx")
        return

    # 检查下载的文件是否为空
    if os.path.getsize("glossary.xlsx") == 0:
        print("Error: Downloaded file is empty")
        # 下载失败，将备份文件名改回glossary
        if os.path.exists(backup_filename):
            os.rename(backup_filename, "glossary.xlsx")
        return


In [50]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog, messagebox, simpledialog
import pandas as pd
from webdav4.client import Client
import os
from getmac import get_mac_address
import pandas as pd
from datetime import datetime
import openpyxl
import webbrowser

username = "imybfu@outlook.com"
password = "axzyqd3a6brx99ei"

def download_from_jianguoyun():
    mac_address = get_mac_address()  # 获取设备的 MAC 地址
    cloud_filename = f'{mac_address.replace(":", "")}.xlsx'
    client = Client(base_url='https://dav.jianguoyun.com/dav/', auth=(username, password))
    # 检查云端文件是否存在
    cloud_path = f'/nutshare/glossary/{cloud_filename}' # 替换为您的实际文件路径
    try:
        file_info = client.exists(cloud_path)
    except HTTPError as e:
        if e.response.status_code == 404:
            print ("Error", "File not found on cloud")
            return
        else:
            print ("Error", f"Failed to check file existence: {str(e)}")
            return
    # 生成当前时间的标记，用于重命名本地文件
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    # 备份本地文件
    backup_filename = f"glossary_{current_time}.xlsx"
    try:
        # 如果本地有glossary文件，将之改名为备份文件
        if os.path.exists("glossary.xlsx"):
            os.rename("glossary.xlsx", backup_filename)
        else:
            print ("Error", "Local file not found")
    except Exception as e:
        print ("Error", f"Failed to backup local file: {str(e)}")
        return
    # 如果文件存在，则下载远程文件到本地
    try:
        client.download_file(from_path=cloud_path, to_path="glossary.xlsx")
        try:
            df = pd.read_excel('glossary.xlsx', engine='openpyxl')  # 重新读取下载后的 glossary.xlsx 文件
            glossary.clear()  # 清空当前的 glossary 字典
            for index, row in df.iterrows():
                term = str(row['terms']).upper()
                full = str(row['full_form'])
                expl = str(row['explanation'])
                if term:
                    glossary[term] = {"full_form": full, "explanation": expl}  # 更新 glossary 字典
        except FileNotFoundError:
            print ("Error", "Failed to load downloaded glossary file")
            remove("glossary.xlsx") # 加载失败时可能产生一个错误的xlsx文件，这时将其删除
            return
        # 如果下载成功，则更新术语列表和标题栏，并显示成功消息
        # update_glossary_list()
        # update_title()
        print ("Success", "File downloaded successfully")
    except Exception as e:
        print ("Error", f"Failed to download file: {str(e)}")
        # 下载失败，将备份文件名改回glossary
        if os.path.exists(backup_filename):
            os.rename(backup_filename, "glossary.xlsx")
        return
download_from_jianguoyun()

Error Local file not found
Success File downloaded successfully
